In [2]:
import numpy as np
import pandas as pd
import zipfile as zp

In [3]:
with zp.ZipFile('train_zsTMYVA.zip','r') as tr, zp.ZipFile('test_9tJUnaB.zip', 'r') as te:
    train = pd.read_csv(tr.open(tr.namelist()[0]))
    test = pd.read_csv(te.open(te.namelist()[0]))

/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (294,299,300,302,307,313,315,365,367,369) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (294,299,300,302,313,315,365,367,369) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
## submit l1 predictions and check score

dart = pd.read_csv('l1/lgbDart_test.csv')
dart['UCIC_ID'] = test['UCIC_ID']
dart.rename(columns={'pred4':'Responders'}, inplace=True)
dart.to_csv('submissions/dart.csv', index=False) # 0.6852367688

In [12]:
tree = pd.read_csv('l1/lgbTree_test.csv')
tree['UCIC_ID'] = test['UCIC_ID']
tree.rename(columns={'pred4':'Responders'}, inplace=True)
tree.to_csv('submissions/tree.csv', index=False) ## 0.6850

In [16]:
reg = pd.read_csv('l1/xgbLR_test.csv')
reg['UCIC_ID'] = test['UCIC_ID']
reg.rename(columns = {'pred4':'Responders'}, inplace=True)
reg.to_csv('submissions/regression.csv', index=False) ## 0.6262

In [23]:
fore = pd.read_csv('l1/xgbRF_test.csv')
fore['UCIC_ID'] = test['UCIC_ID']
fore.rename(columns = {'pred4':'Responders'}, inplace=True)
fore.to_csv('submissions/forest.csv', index=False) ## 0.6819

In [24]:
xgtree = pd.read_csv('l1/xgbTree_test.csv')
xgtree['UCIC_ID'] = test['UCIC_ID']
xgtree.rename(columns = {'pred4':'Responders'}, inplace=True)
xgtree.to_csv('submissions/xgtree.csv', index=False) ## 0.6808

In [29]:
### simple mean
cal = (dart['Responders'] + tree['Responders'] + reg['Responders'] + fore['Responders'] + xgtree['Responders'])/5

In [33]:
aver = pd.DataFrame({'UCIC_ID':test['UCIC_ID'], 'Responders':cal})
aver.to_csv('submissions/bagging_l1_preds.csv', index=False) #0.68535

In [37]:
### weighted_average
wei = (dart['Responders']*0.30 + tree['Responders']*0.30 + reg['Responders']*0.05 + fore['Responders']*0.20 + xgtree['Responders']*0.15)

In [38]:
we_ave = pd.DataFrame({'UCIC_ID':test['UCIC_ID'], 'Responders':wei})
we_ave.to_csv('submissions/weiave_l1_preds.csv', index=False) ## 0.6852

In [41]:
## geometric mean
from scipy.stats.mstats import gmean

In [58]:
res = pd.DataFrame({'r1':dart['Responders'],'r2':tree['Responders'],'r3':reg['Responders'],'r4':fore['Responders'], 'r5':xgtree['Responders']})

In [59]:
res['Responders'] = gmean(res, axis=1)

In [61]:
res = pd.DataFrame({'UCIC_ID':test['UCIC_ID'], 'Responders':res['Responders']})

In [63]:
res.to_csv('submissions/geomean_l1_preds.csv', index=False) # 0.68431

### Lets build models on L1 preds

In [168]:
train = pd.read_pickle('papa_train.pkl')
test = pd.read_pickle('papa_test.pkl')

In [173]:
test = test.reset_index(drop=True)

In [177]:
## add predictions in train and test
train['pred_dart'] = pd.read_csv('l1/lgbDart_train.csv', usecols=['Responders'])['Responders']
train['pred_tree'] = pd.read_csv('l1/lgbTree_train.csv', usecols=['Responders'])['Responders']
train['pred_reg'] = pd.read_csv('l1/xgbLR_train.csv', usecols=['Responders'])['Responders']
train['pred_fore'] = pd.read_csv('l1/xgbRF_train.csv', usecols=['Responders'])['Responders']
train['pred_xgtree'] = pd.read_csv('l1/xgbTree_train.csv', usecols=['Responders'])['Responders']

In [178]:
test['pred_dart'] = dart['Responders']
test['pred_tree'] = tree['Responders']
test['pred_reg'] =  reg['Responders']
test['pred_fore'] = fore['Responders']
test['pred_xgtree'] = xgtree['Responders']

In [208]:
## write pred data separately
train_pred = train.loc[:,['pred_dart','pred_tree','pred_reg','pred_fore','pred_xgtree']]
test_pred = test.loc[:,['pred_dart','pred_tree','pred_reg','pred_fore','pred_xgtree']]

In [211]:
train_pred.to_csv('train_pred.csv', index=False)
test_pred.to_csv('test_pred.csv', index=False)

In [179]:
train.head()

,AGRI_Closed_PrevQ1,AGRI_DATE,AGRI_PREM_CLOSED_PREVQ1,AGRI_TAG_LIVE,AL_CNC_Closed_PrevQ1,AL_CNC_DATE,AL_CNC_PREM_CLOSED_PREVQ1,AL_CNC_TAG_LIVE,AL_Closed_PrevQ1,AL_DATE,...,pca_10,tsvd_10,ica_10,grp_10,srp_10,pred_dart,pred_tree,pred_reg,pred_fore,pred_xgtree
0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.463040,-1.258798,0.001182,-0.546541,-5.128583,0.298309,0.311703,0.500039,0.321626,0.282638
1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,4.736117,4.788461,0.001884,-28.458333,13.682360,0.121993,0.073490,0.018189,0.117408,0.105152
2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.173977,0.514187,-0.000799,3.003399,-3.222910,0.845566,0.902196,0.613327,0.860199,0.923288
3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.616949,0.631813,0.001642,0.466489,4.872285,0.072438,0.055597,0.071254,0.065348,0.051623
4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,3.217670,3.274776,-0.002916,-0.842176,-7.854273,0.396067,0.450381,0.213564,0.440154,0.430313


In [180]:
test.head()

,AGRI_Closed_PrevQ1,AGRI_DATE,AGRI_PREM_CLOSED_PREVQ1,AGRI_TAG_LIVE,AL_CNC_Closed_PrevQ1,AL_CNC_DATE,AL_CNC_PREM_CLOSED_PREVQ1,AL_CNC_TAG_LIVE,AL_Closed_PrevQ1,AL_DATE,...,pca_10,tsvd_10,ica_10,grp_10,srp_10,pred_dart,pred_tree,pred_reg,pred_fore,pred_xgtree
0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.066166,-0.016615,-0.002190,-9.721923,0.425465,0.335778,0.350223,0.469787,0.327790,0.300304
1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.313483,0.318584,-0.002919,4.005902,-0.639006,0.028943,0.023414,0.060528,0.028299,0.029012
2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.665584,0.589714,0.002582,6.341306,3.805967,0.045216,0.039772,0.082668,0.041646,0.072659
3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,3.136754,3.201557,-0.000415,-3.788141,-3.351600,0.155093,0.155252,0.439493,0.156464,0.140133
4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,2.189452,1.852458,0.000418,-1.909757,1.098223,0.168107,0.222249,0.324906,0.223561,0.242268


### Running LGBM

In [181]:
feature_names = [col for col in train.columns if col not in ['UCIC_ID','Responders']]
target = train['Responders']

In [182]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [183]:
X_train, X_valid, y_train, y_valid = train_test_split(train, target, test_size = 0.3, stratify = target)

In [184]:
ltrain = lgb.Dataset(X_train[feature_names], label=y_train)
lvalid = lgb.Dataset(X_valid[feature_names], label=y_valid)

In [196]:
paramslgb = {  
    'learning_rate':0.1,
    'max_depth':7,
    'boosting':'dart',
    'objective':'binary',
    'metric':'auc',
    'seed':2017,
    'feature_fraction':0.9,
    'bagging_fraction':0.9,
    'num_leaves':30
#     'lambda_l1':16,
#     'lambda_l2':16
}

In [197]:
mod1 = lgb.train(paramslgb, ltrain, num_boost_round=1000, valid_sets=lvalid, verbose_eval=20, early_stopping_rounds=40)

Train until valid scores didn't improve in 40 rounds.
[20]	valid_0's auc: 0.87213
[40]	valid_0's auc: 0.87231
[60]	valid_0's auc: 0.872188
Early stopping, best iteration is:
[38]	valid_0's auc: 0.872319


In [198]:
ld_train = lgb.Dataset(train[feature_names], target)

In [199]:
mod2 = lgb.train(paramslgb, ld_train, num_boost_round = mod1.best_iteration, valid_sets=ld_train, verbose_eval = 20)

[20]	training's auc: 0.872599


In [200]:
pred = mod2.predict(test[feature_names])

In [202]:
pred[:10]

array([ 0.27192448,  0.09678881,  0.0994129 ,  0.16086624,  0.17654175,
        0.09678881,  0.10565129,  0.09678881,  0.12271422,  0.09856992])

In [203]:
sub = pd.read_csv('sample_submission_fBo3EW5.csv')
sub['UCIC_ID'] = test['UCIC_ID']

In [205]:
sub['Responders'] = pred

In [206]:
sub.head()

,UCIC_ID,Responders
0,337734,0.271924
1,488166,0.096789
2,410785,0.099413
3,389145,0.160866
4,221090,0.176542


In [207]:
sub.to_csv('submissions/l1_lgb_pred.csv', index=False) ## l1_Stack

### RUN XGB

In [142]:
import xgboost as xgb

/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [141]:
## default
params = {"objective": "binary:logistic",
          "booster": "gbtree",
          "nthread": 4,
          "eta": 0.1, # 0.1
          "max_depth": 5, # 7
          "subsample": 0.9,
          "colsample_bytree": 0.7,
          "seed": 2016, 
         "eval_metric":"auc"}

In [143]:
dtrain = xgb.DMatrix(X_train[feature_names], y_train, missing=-1)
dvalid = xgb.DMatrix(X_valid[feature_names], y_valid, missing=-1)
dtest = xgb.DMatrix(test[feature_names])

In [144]:
watchlist = [(dtrain, 'train'),(dvalid, 'valid')]

In [145]:
clf1 = xgb.train(params, dtrain, num_boost_round=1000, evals=watchlist, maximize=True, verbose_eval=20, early_stopping_rounds=40)

[0]	train-auc:0.869443	valid-auc:0.870315
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 40 rounds.
[20]	train-auc:0.87398	valid-auc:0.872451
[40]	train-auc:0.876778	valid-auc:0.872416
[60]	train-auc:0.879685	valid-auc:0.872434
[80]	train-auc:0.881124	valid-auc:0.872345
Stopping. Best iteration:
[48]	train-auc:0.878046	valid-auc:0.872561



In [146]:
predX1 = clf1.predict(dtest)

In [154]:
predX1[:10]

array([ 0.18015744,  0.02319355,  0.02166098,  0.0507172 ,  0.07527688,
        0.02685545,  0.0537979 ,  0.02310515,  0.02427379,  0.01624846], dtype=float32)

In [150]:
## now prediction on whole data
wdtrain = xgb.DMatrix(data=train[feature_names], label=target, missing=-1)

In [151]:
clf2 = xgb.train(params, wdtrain, num_boost_round=clf1.best_iteration)

In [152]:
predX2 = clf2.predict(dtest)

In [153]:
predX2[:10]

array([ 0.08134915,  0.02921624,  0.03544   ,  0.03894316,  0.06571756,
        0.02051844,  0.03412097,  0.0323839 ,  0.0201434 ,  0.02030157], dtype=float32)

In [155]:
sub1 = pd.read_csv('sample_submission_fBo3EW5.csv')
sub1['UCIC_ID'] = test['UCIC_ID']
sub1['Responders'] = predX1

In [156]:
sub2 = pd.read_csv('sample_submission_fBo3EW5.csv')
sub2['UCIC_ID'] = test['UCIC_ID']
sub2['Responders'] = predX2

In [157]:
sub1.to_csv('submissions/xgb_l1_predX1.csv', index=False) ## 0.30
sub2.to_csv('submissions/xgb_l1_predX2.csv', index=False) ## 0.3376

### XGB Linear

In [158]:
set_params = {"objective": "binary:logistic",
                      "booster": "gblinear",
                      "nthread": 4,
                      "alpha":4,
                      "lambda":2,
                      "seed": 2016, 
                      "eval_metric":"auc",
                      "silent":1}

In [159]:
clf1 = xgb.train(set_params, dtrain, num_boost_round=1000, evals=watchlist, maximize=True, verbose_eval=20, early_stopping_rounds=40)

[0]	train-auc:0.836541	valid-auc:0.838063
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 40 rounds.
[20]	train-auc:0.87039	valid-auc:0.870573
[40]	train-auc:0.870677	valid-auc:0.87079
[60]	train-auc:0.870731	valid-auc:0.8708
[80]	train-auc:0.870759	valid-auc:0.870776
Stopping. Best iteration:
[46]	train-auc:0.87071	valid-auc:0.870827



In [160]:
predX1 = clf1.predict(dtest)